In [2]:
import numpy as np
import pandas as pd
import random
import os
import glob
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization
import lightgbm as lgb

In [3]:
train_df=pd.read_csv("/content/drive/MyDrive/DACON/태양광/train_reshape.csv")
test_df=pd.read_csv("/content/drive/MyDrive/DACON/태양광/test_reshape.csv")

In [4]:
train_df_1=train_df.iloc[:365*48,:]

from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(train_df_1[train_df_1.columns[~train_df_1.columns.isin(['TARGET_day7','TARGET_day8'])]], train_df_1.loc[:, 'TARGET_day7'], test_size=0.3, random_state=42)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(train_df_1[train_df_1.columns[~train_df_1.columns.isin(['TARGET_day7','TARGET_day8'])]], train_df_1.loc[:, 'TARGET_day8'], test_size=0.3, random_state=42)

X = train_df_1.drop(columns=['TARGET_day7','TARGET_day8'])
y_1 = train_df_1['TARGET_day7']
y_2 = train_df_1['TARGET_day8']

In [5]:
quantiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for i in quantiles:
  print('Quatiles:',i)
  def bayes_parameter_opt_lgb(X, y, alpha=i, init_round=5, opt_round=30, n_folds=3, n_estimators=5000, random_seed=42, output_process=True):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, min_data_in_leaf,alpha=alpha):
        params = {'objective':'quantile','alpha':alpha,'boosting_type': 'dart'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=False, verbose_eval =200,metrics='quantile')

        return -min(cv_result['quantile-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.001, 1.0),
                                            'num_leaves': (16, 1024),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 30),
                                            'min_data_in_leaf': (16, 1024)}, random_state=random_seed,verbose=0)

    
    #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    models=[]
    for model in range(len( lgbBO.res)):
        models.append(lgbBO.res[model]['target'])
    
    
    # return best parameters
    return lgbBO.res[pd.Series(models).idxmax()]['target'],lgbBO.res[pd.Series(models).idxmax()]['params']

  print('BayesianOptimization is working')
  opt_params_1 = bayes_parameter_opt_lgb(X, y_1, init_round=5, opt_round=30, n_folds=3, random_seed=42)
  print('BayesianOptimization is done')
  opt_params_1[1]["num_leaves"] = int(round(opt_params_1[1]["num_leaves"]))
  opt_params_1[1]['max_depth'] = int(round(opt_params_1[1]['max_depth']))
  opt_params_1[1]['min_data_in_leaf'] = int(round(opt_params_1[1]['min_data_in_leaf']))
  opt_params_1 = opt_params_1[1]
  opt_params_1

  q=i
    # Get the model and the predictions in (a) - (b)
  def LGBM_2(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    params = opt_params_1
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile',boosting_type='dart', alpha=q,  n_estimators=5000, **params, random_state=42)                   
                                             
    model.fit(X_train, Y_train, eval_metric = ['quantile'], 
    eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300, verbose=500)

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

    # Target 예측

  def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    pred , model = LGBM_2(q, X_train, Y_train, X_valid, Y_valid, X_test)
    LGBM_models.append(model)
    LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=[i]
    
    return LGBM_models, LGBM_actual_pred

      # Target2
  models, results = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, test_df)

  results.to_csv("/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/1year/year1_results_{}.csv".format(i), index=False)

Quatiles: 0.1
BayesianOptimization is working
BayesianOptimization is done
[500]	valid_0's quantile: 1.24075
[1000]	valid_0's quantile: 1.20846
[1500]	valid_0's quantile: 1.19926
[2000]	valid_0's quantile: 1.19074
[2500]	valid_0's quantile: 1.18994
[3000]	valid_0's quantile: 1.18445
[3500]	valid_0's quantile: 1.18379
[4000]	valid_0's quantile: 1.17921
[4500]	valid_0's quantile: 1.17658
[5000]	valid_0's quantile: 1.17516
Quatiles: 0.2
BayesianOptimization is working
BayesianOptimization is done
[500]	valid_0's quantile: 1.81186
[1000]	valid_0's quantile: 1.75597
[1500]	valid_0's quantile: 1.73967
[2000]	valid_0's quantile: 1.73307
[2500]	valid_0's quantile: 1.72947
[3000]	valid_0's quantile: 1.72424
[3500]	valid_0's quantile: 1.71825
[4000]	valid_0's quantile: 1.71549
[4500]	valid_0's quantile: 1.71221
[5000]	valid_0's quantile: 1.70886
Quatiles: 0.3
BayesianOptimization is working
BayesianOptimization is done
[500]	valid_0's quantile: 2.07915
[1000]	valid_0's quantile: 2.00262
[1500]	v

In [ ]:
test_files_1  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/1year/*csv'))
test_files_2  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/2year/*csv'))
test_files_3  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/*csv'))

result_1 = []
result_1 = pd.DataFrame()

for i in range(0,len(test_files_1)):
  result_1 = pd.concat([result_1, pd.read_csv(test_files_1[i])], axis=1)

result_2 = []
result_2 = pd.DataFrame()

for i in range(0,len(test_files_2)):
  result_2 = pd.concat([result_2, pd.read_csv(test_files_2[i])], axis=1)

result_3 = []
result_3 = pd.DataFrame()

for i in range(0,len(test_files_3)):
  result_3 = pd.concat([result_3, pd.read_csv(test_files_3[i])], axis=1)

result_1.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
result_2.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
result_3.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']

In [ ]:
day_1_res = []
day_1_res = pd.DataFrame(columns=['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9'])

for i in result_1.columns:
  day_1_res[i] = (result_1[i] + result_2[i] + result_3[i]) / 3

display(day_1_res)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.090000
1,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.093333
2,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.090000
3,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.090000
4,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.083333
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.010000
3884,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.010000
3885,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.013333
3886,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.013333


In [ ]:
def to_zero(x):
  if x < 0:
    return 0
  else:
    return x

for i in result_1.columns:
  day_1_res[i] = day_1_res[i].apply(to_zero)

display(day_1_res)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.090000
1,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.093333
2,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.090000
3,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.090000
4,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.083333
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.010000
3884,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.010000
3885,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.013333
3886,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.013333
